In [106]:
from IPython.core.debugger import set_trace
import azure.mgmt.consumption
import adal
from msrestazure.azure_active_directory import AADTokenCredentials
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

In [107]:
# Parameters need for API
subscription = os.getenv("AZURE_SUBSCRIPTION_ID")
tenant = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")

In [108]:
def authenticate_client_key(tenant, client_id, client_secret):
    """
    Authenticate using service principal w/ key.
    """
    authority_host_uri = 'https://login.microsoftonline.com'
    authority_uri = authority_host_uri + '/' + tenant
    resource_uri = 'https://management.core.windows.net/'
    
    context = adal.AuthenticationContext(authority_uri, api_version=None)
    mgmt_token = context.acquire_token_with_client_credentials(resource_uri, client_id, client_secret)
    credentials = AADTokenCredentials(mgmt_token, client_id)

    return credentials

In [109]:
credentials = authenticate_client_key(tenant, client_id, client_secret)

In [110]:
client = azure.mgmt.consumption.ConsumptionManagementClient(credentials, subscription)

In [112]:
client.operations.list()

In [113]:
for op in client.operations.list():
    print(op.__dict__)

{'additional_properties': {'id': 'Microsoft.Consumption/usageDetails/read'}, 'name': 'Microsoft.Consumption/usageDetails/read', 'display': <azure.mgmt.consumption.models.operation_display.OperationDisplay object at 0x1074d19e8>}
{'additional_properties': {'id': 'Microsoft.Consumption/reservationSummaries/read'}, 'name': 'Microsoft.Consumption/reservationSummaries/read', 'display': <azure.mgmt.consumption.models.operation_display.OperationDisplay object at 0x1074d1a58>}
{'additional_properties': {'id': 'Microsoft.Consumption/reservationDetails/read'}, 'name': 'Microsoft.Consumption/reservationDetails/read', 'display': <azure.mgmt.consumption.models.operation_display.OperationDisplay object at 0x1074d1ac8>}
{'additional_properties': {'id': 'Microsoft.Consumption/reservationTransactions/read'}, 'name': 'Microsoft.Consumption/reservationTransactions/read', 'display': <azure.mgmt.consumption.models.operation_display.OperationDisplay object at 0x1074d1b38>}
{'additional_properties': {'id': '

In [128]:
details = []
for detail in client.usage_details.list_by_billing_period(billing_period_name='201808-1', expand='properties/additionalProperties'):
    details.extend([detail.__dict__])

In [129]:
import pandas as pd
import numpy as np
df = pd.DataFrame(details)

In [130]:
df['billing_period_id'][0]

'/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/billingPeriods/201808-1'

In [132]:
df

,account_name,additional_properties,billable_quantity,billing_period_id,consumed_service,cost_center,currency,department_name,id,instance_id,...,name,pretax_cost,product,subscription_guid,subscription_name,tags,type,usage_end,usage_quantity,usage_start
0,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,1479f5be-37a8-18e6-945a-b7b242e90d18,0.00032160000000000001085798118083403096534311...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.00080400000000000002714495295208507741335779...,2018-06-12 00:00:00+00:00
1,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,89f49323-b9cb-aa79-d2c7-c6d6a4c6537d,1.79999999999999997148616048998903682587524599...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.00050000000000000001040834085586084256647154...,2018-06-12 00:00:00+00:00
2,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,899d2e80-03ff-b7ac-6adc-88d882b300fa,0,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,1.61960299999999990383514614222804084420204162...,2018-06-12 00:00:00+00:00
3,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,f6b1f540-fd3d-57a2-a471-6faaba52c3e5,0.32047307600000002336670945624064188450574874...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.76668199999999997462651890600682236254215240...,2018-06-12 00:00:00+00:00
4,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,2defb18a-fb8f-35fb-9ba3-9338950be88f,0,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.05984099999999999835997854802371875848621129...,2018-06-12 00:00:00+00:00
5,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,8301af12-ad04-9331-7d45-585295d95b5f,3.59999999999999981062342297149364567232510125...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.00010000000000000000479217360238592959831294...,2018-06-12 00:00:00+00:00
6,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,05f3c91a-c01b-7f34-b54f-dc1d8dc49a45,0.02737718999999999916483695017177524277940392...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.00138900000000000004365952044338428095215931...,2018-06-12 00:00:00+00:00
7,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,3bfd4d04-67b4-3420-5c54-03da08593c99,0.00000176399999999999996676248135984987897018...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-06-13 00:00:00+00:00,0.00489999999999999984179321899091519298963248...,2018-06-12 00:00:00+00:00
8,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-

In [133]:
df['pretax_cost_rounded'] = df['pretax_cost'].astype(float).round(2)

In [134]:
df.billing_period_id.unique()

array(['/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/billingPeriods/201808-1'],
      dtype=object)

In [137]:
df.groupby(by = ['billing_period_id'])[['pretax_cost','pretax_cost_rounded']].sum()

,pretax_cost_rounded
billing_period_id,
/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/billingPeriods/201808-1,109.51


In [102]:
df

,account_name,additional_properties,billable_quantity,billing_period_id,consumed_service,cost_center,currency,department_name,id,instance_id,...,pretax_cost,product,subscription_guid,subscription_name,tags,type,usage_end,usage_quantity,usage_start,pretax_cost_rounded
0,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.00001108799999999999954906904076068485665018...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.03080000000000000098809849191638932097703218...,2018-07-12 00:00:00+00:00,0.00
1,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.00001728000000000000057330008795819509259672...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.00014400000000000000364812346997922531954827...,2018-07-12 00:00:00+00:00,0.00
2,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.00022279999999999999457656052470611030003055...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.00055699999999999998644140131176527575007639...,2018-07-12 00:00:00+00:00,0.00
3,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.00062399999999999998870348072443903220118954...,2018-07-12 00:00:00+00:00,0.00
4,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.00012000000000000000304010289164935443295689...,2018-07-12 00:00:00+00:00,0.00
5,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.00132900000000000010312584119986922814860008...,2018-07-12 00:00:00+00:00,0.00
6,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.00000676800000000000025277696602543642256932...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.01880000000000000073829831137572909938171505...,2018-07-12 00:00:00+00:00,0.00
7,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.00001281600000000000062283858626166477279184...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.03559999999999999970023978335120773408561944...,2018-07-12 00:00:00+00:00,0.00
8,None,None,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,None,None,USD,None,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,...,0.63576575999999995758571458281949162483215332...,None,3e6b71a1-1c47-4188-a4dc-793259a87549,None,None,Microsoft.Consumption/usageDetails,2018-07-13 00:00:00+00:00,0.03225599999999999994981791928694292437285184...,2

In [40]:
# Only EA subscriptions are supported for this request
sheet = client.price_sheet.get()

ErrorResponseException: (404) No subscriptions were found for guid(s) 3e6b71a1-1c47-4188-a4dc-793259a87549. Only EA subscriptions are supported for this request. Please ensure that offer type of the subscription is either AZR-0017P or AZR-0148P. (Request ID: 02f1c60d-5017-4898-bdad-58616840f326)